# User Management
* This notebook covers the following:
    * create / update / revoke invite
    * query for active invites
    * query for remaining allowed invites to an organization
    * query for remaining available seats in an organization
    * set and update organization roles
    * assign users to projects
        * set / update / revoke project role
    * delete users from org
* User management is currently a beta feature so any query with an experimental warning could potentially be updated. 

In [1]:
!pip install labelbox

In [2]:
from labelbox import Project, Dataset, Client, User
from labelbox.schema.organization import ProjectRole
from getpass import getpass
import os

In [3]:
# If you don't want to give google access to drive you can skip this cell
# and manually set `API_KEY` below.

COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !pip install colab-env -qU
    from colab_env import envvar_handler
    envvar_handler.envload()

API_KEY = os.environ.get("LABELBOX_API_KEY")
if not os.environ.get("LABELBOX_API_KEY"):
    API_KEY = getpass("Please enter your labelbox api key")
    if COLAB:
        envvar_handler.add_env("LABELBOX_API_KEY", API_KEY)

In [4]:
ENDPOINT = "https://api.labelbox.com/graphql"
client = Client(api_key=API_KEY, endpoint=ENDPOINT)
organization = client.get_organization()

In [5]:
# Please provide a dummy email here:
# Preferrably one you can access. If you have a google account you can do email+1@<domain>.com
DUMMY_EMAIL = "< SET THIS >"
# This should be set to an account that you wan't to change the permissions for.
# You could invite a new user, accept the invite and use that account if you don't want to effect any active users
DUMMY_USER_ACCOUNT_ID = "ckneh4n8c9qvq0706uwwg5i16"

### Roles
* When inviting a new user to an organization, there are various roles to select from.
* All available roles to your org can be accessed via `client.get_roles()`

In [6]:
roles = client.get_roles()
for role in roles:
    print(role.name, ":", role.uid)

ADMIN : cjlvi91a41aab0714677xp87h
REVIEWER : cjlvi919b1aa50714k75euii5
TEAM_MANAGER : cjlvi919q1aa80714b7z3xuku
LABELER : cjlvi914y1aa20714372uvzjv
NONE : cjmb6xy80f5vz0780u3mw2rj4


* Above we printed out all of the roles available to the current org.
* Notice the `NONE`. That is for project level roles

In [7]:
# Roles can be accessed either way:
print(roles['LABELER'])
print(roles.LABELER)
# And, as shown above, they are iterable.

<Role {'name': 'LABELER', 'uid': 'cjlvi914y1aa20714372uvzjv'}>
<Role {'name': 'LABELER', 'uid': 'cjlvi914y1aa20714372uvzjv'}>


### Create
* Users are created by sending an invite
* An email will be sent to them and they will be asked to join your organization

#### Organization Level Permissions
* Invite a new labeler with labeling permissions on all projects

In [8]:
# First make sure that you have enough seats:
organization.invite_limit()

Experimental queries/mutations aren't guarenteed to maintain their interface.


InviteLimit(remaining=2, used=3, limit=5)

In [9]:
invite = organization.invite_user(DUMMY_EMAIL, roles.LABELER)

Experimental queries/mutations aren't guarenteed to maintain their interface.
Experimental queries/mutations aren't guarenteed to maintain their interface.
Experimental queries/mutations aren't guarenteed to maintain their interface.


In [10]:
print(invite.created_at)
print(invite.organization_role_name)
print(invite.email)
invite.revoke()  #Delete explained more below

Experimental queries/mutations aren't guarenteed to maintain their interface.


2021-04-12 18:15:56.440000+00:00
Labeler
< SET THIS >


#### Project Level Permissions
* Invite a new labeler with labeling permissions specific to a set of projects
* Here we set organization level permissions to Roles.NONE to indicate that the user only has project level permissions

In [11]:
project = client.create_project(name="test_user_management")
project_role = ProjectRole(project=project, role=roles.REVIEWER)
invite = organization.invite_user(DUMMY_EMAIL,
                                  roles.NONE,
                                  project_roles=[project_role])

Experimental queries/mutations aren't guarenteed to maintain their interface.
Experimental queries/mutations aren't guarenteed to maintain their interface.
Experimental queries/mutations aren't guarenteed to maintain their interface.


### Read

In [12]:
invites = list(organization.invites())
print(invites)
print("EMAIL : ", invites[0].email)

Experimental queries/mutations aren't guarenteed to maintain their interface.


[<Invite ID: cknex2yj9006n0y4wdamicqfp>]
EMAIL :  < SET THIS >


In [13]:
users = list(organization.users())
print(users[0])

<User {'created_at': datetime.datetime(2021, 1, 20, 1, 2, 31, tzinfo=datetime.timezone.utc), 'email': 'msokoloff@labelbox.com', 'intercom_hash': '050b8b292bd66abf88ea9a279c6ee80a530e0f9cf98a06e1d5372ef13697f46b', 'is_external_user': False, 'is_viewer': True, 'nickname': 'msokoloff', 'name': 'Matt Sokoloff', 'picture': 'https://lh6.googleusercontent.com/-gO94Mcw4PGs/AAAAAAAAAAI/AAAAAAAAAAA/AMZuuclv_5P5WICzV1aImhxGADj_OS5c7w/s96-c/photo.jpg', 'uid': 'ckk4q1vgwc0vp0704xhx7m4vk', 'updated_at': datetime.datetime(2021, 4, 1, 15, 48, 27, tzinfo=datetime.timezone.utc)}>


In [14]:
# We can also see which users are invited to a project
for invite in project.invites():
    print(invite.project_roles)

Experimental queries/mutations aren't guarenteed to maintain their interface.


[ProjectRole(project=<Project ID: cknex2qzhoyar0790cckzc7jv>, role=<Role ID: cjlvi919b1aa50714k75euii5>)]


* Available seats remaining in the organization

In [15]:
user_limit = organization.user_limit()
print("Seats remaining : ", user_limit.remaining)
print("Seats used : ", user_limit.used)
print("Max users allowed :", user_limit.limit)

Experimental queries/mutations aren't guarenteed to maintain their interface.


Seats remaining :  2
Seats used :  3
Max users allowed : 5


### Update
* There is no update on invites. Instead you must delete and resend them
* You can update User roles

In [16]:
user = client._get_single(User, DUMMY_USER_ACCOUNT_ID)

# Give the user organization level permissions
user.update_org_role(roles.LABELER)
print(user.org_role())
# Restore project level permissions
user.update_org_role(roles.NONE)
print(user.org_role())
# Make the user a labeler for the current project
user.upsert_project_role(project, roles.LABELER)
print(user.org_role())

<OrgRole {'name': 'Labeler', 'uid': 'cjlvi914y1aa20714372uvzjv'}>
<OrgRole {'name': 'None', 'uid': 'cjmb6xy80f5vz0780u3mw2rj4'}>
<OrgRole {'name': 'None', 'uid': 'cjmb6xy80f5vz0780u3mw2rj4'}>


In [17]:
# Remove the user from a project (Same as setting the project role to `roles.NONE`)
user.remove_from_project(project)

### Delete

In [18]:
# This deletes the user from the org.
# You will have to reinvite them
invite.revoke()

Experimental queries/mutations aren't guarenteed to maintain their interface.


* Delete the User
* Make sure you want to remove the user from the org:
* `>>> organization.remove_user(user)`

### Cleanup
* We created an extra project. Let's delete it

In [19]:
project.delete()